In [1]:
!python --version

Python 3.9.12


In [19]:
# Imports
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error



In [3]:
# Import Data
train = pd.read_parquet('/workspaces/mlops_zoomcamp/data/yellow_tripdata_2023-01.parquet')

In [4]:
train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
#Q1
len(train.columns)

19

In [6]:
#Q2
# Cast datetime columns
train.tpep_pickup_datetime = pd.to_datetime(train.tpep_pickup_datetime)
train.tpep_dropoff_datetime = pd.to_datetime(train.tpep_dropoff_datetime)

# Compute duration
train["duration"] = train.tpep_dropoff_datetime - train.tpep_pickup_datetime
train.duration = train.duration.apply(lambda td: td.total_seconds()/60)

# Compute std
train.duration.describe()


count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [7]:
# Q3 DROPPING OUTLIERS
print(((train["duration"] >= 1) & (train["duration"] <= 60)).mean())

# Drop
train = train[((train["duration"] >= 1) & (train["duration"] <= 60))]




0.9812202822125979


In [8]:
# Q4
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

In [9]:
# Cast columns as str
train[categorical] = train[categorical].astype(str)

In [10]:
# Transform df to dictionaries
X_train_dicts = train[categorical].to_dict(orient = 'records')

In [11]:
# build dict vectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(X_train_dicts)


In [12]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [13]:
len(dv.feature_names_)

515

In [ ]:
#5


In [15]:
y_train = train["duration"]

In [16]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [20]:
y_pred = reg.predict(X_train)

In [22]:
from math import sqrt
sqrt(mean_squared_error(y_train,y_pred))

7.649261027806873

In [23]:
#6
test = pd.read_parquet("/workspaces/mlops_zoomcamp/data/yellow_tripdata_2023-02.parquet")

In [24]:
# Cast datetime columns
test.tpep_pickup_datetime = pd.to_datetime(test.tpep_pickup_datetime)
test.tpep_dropoff_datetime = pd.to_datetime(test.tpep_dropoff_datetime)

# Compute duration
test["duration"] = test.tpep_dropoff_datetime - test.tpep_pickup_datetime
test.duration = test.duration.apply(lambda td: td.total_seconds()/60)

In [25]:
# Cast columns as str
test[categorical] = test[categorical].astype(str)

In [26]:
# Transform df to dictionaries
X_test_dicts = test[categorical].to_dict(orient = 'records')
X_test = dv.transform(X_test_dicts)

In [27]:
y_test = test["duration"]

In [28]:
y_pred = reg.predict(X_test)

In [30]:
sqrt(mean_squared_error(y_test,y_pred))

42.28477445011177